In [1]:
from datasets import load_dataset

In [4]:
dataset = load_dataset("yelp_review_full")

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
dataset["train"][111]

{'label': 2,
 'text': "As far as Starbucks go, this is a pretty nice one.  The baristas are friendly and while I was here, a lot of regulars must have come in, because they bantered away with almost everyone.  The bathroom was clean and well maintained and the trash wasn't overflowing in the canisters around the store.  The pastries looked fresh, but I didn't partake.  The noise level was also at a nice working level - not too loud, music just barely audible.\\n\\nI do wish there was more seating.  It is nice that this location has a counter at the end of the bar for sole workers, but it doesn't replace more tables.  I'm sure this isn't as much of a problem in the summer when there's the space outside.\\n\\nThere was a treat receipt promo going on, but the barista didn't tell me about it, which I found odd.  Usually when they have promos like that going on, they ask everyone if they want their receipt to come back later in the day to claim whatever the offer is.  Today it was one of th

In [6]:
dataset.shape

{'train': (650000, 2), 'test': (50000, 2)}

In [7]:
dataset.column_names

{'train': ['label', 'text'], 'test': ['label', 'text']}

In [8]:
##  
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [9]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(dataset["train"])

,label,text
0,4 stars,"I love this place. There, I said it.\n\nIf I could pick a secret super-hero dining tool, it would be an extending fork. I love to try everything - all of it. A perfect meal for me means I have sampled it all.... Shared with my neighbor and snagged a little something from your plate. No need here, small plates built to share means people like me are happy campers!\n\nAlthough its billed as a Spanish tapas joint, it has a more American flare.\n\nMy faves of the evening:\n- goat cheese lollipops - ohhhh we ordered more!\n- steak\n- donuts\n- churros (clearly I have a dessert problem)\n\nMy not faves -\n- calamari - table loved it, just not my thing\n- paella - my muscles were a bit strong\n\nFantastic drinks, people watching and service. We dines with 8 people, and had a ton of dishes. The bill slit was 120 which seemed pretty good. Great value, and super convenient for the cirque show.\n\nIt didn't hurt that we sat next to a fabulous bachelorette party dressed as Air Force pilots (nice one ladies)\n\nIll go back. Loads of fun and great food."
1,5 stars,Awesome saloon! Went to this place for a after work get together! I may have used coupons to get my food and beer although it did not take away my bull riding experience! Watched YouTube videos for a few days on how to ride a bull but nothing can prepare you for this Beastly Bohemeth! I paid $40 and signed my life away and got to ride the Bull! I figured how many opportunities am I going to get riding a bull!?!? Awesome staff and supportive staff and bull fighters!
2,3 stars,"Had a \""get $20 worth of food for $10\"" offer online, so I wanted to try this place.\n\nGonna have to give this place a big ole' \""Meh.\""\n\nCompared to Playa Hermosa, this joint is definitely a 3 to their 5.\n\nI had the Molcajete Mariscos (which I also had a Playa) to compare the two. Playa's version is MUCH tastier and has more food in it. Less \""gloppy.\""\n\nThe service was a little disjointed (for which I don't reduce stars unless it falls into the TERRIBLE range) in that I had to ask her twice for a couple of things, and she was not overloaded. My requests seemed to just fall on the backside of remembrance.\n\nThe $2 Happy Hour Maggies were decent tasting, although small-ish when you consider the total quantity of fluid. They were well-spiked though, as I did get a little buzz from having just two of them.\n\nThe salsa and chips were above-average I think - the salsa both red and a little smoky and chunky. Chips on the thick side.\n\nI'd try them again, just to see the range of the offerings.\n\nAs people who follow my reviews will notice, I did indeed follow my rule of never giving a highly-rated place 1 or 2 stars after just one visit. Can't have a full knowledge of all that goes on there from just a single visit, IMHO."
3,1 star,"Expensive!!!!\nThat is the only word I have for Ella's. We ate here on a Saturday for lunch. It had great ambiance and the kids loved it. There were 4 of us. We all had sandwiches and my cousin and I shared a $12 sandwich plate with all the fixins. My wife and his girlfriend had sandwiches and we ordered 3 drinks and a water. The kids ate off of our plates as they always do. I figured the bill was going to be around $40 because we didn't even order ice cream. 3 drinks, 2 sandwiches and 1 \""shared sandwich\"" and our bill was $61.62. WTH?!?!?!?\n\nI went over it with a fine-tooth comb and noticed they charge almost $5 for a small order of cheese fries. The drinks were $3 each and their sandwiches were over $10 each. WTH?!?!?!?!?\n\nI couldn't believe how much they charge for cold meat sandwiches. We could have spend $40 at Panera and had more food and much higher quality.\n\nIt looks like this place is getting a lot of bad reviews lately. I really hope they are reading their Yelp page and they fix things. I will never go back here again. I can eat at a nice steak house for $60. Cold meat sandwiches aren't worth as much as this place charges for t

In [11]:
### 数据处理
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [14]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,5 stars,Ok this place was soo fantastic. I ordered the appetizer Sicilian garlic cheese bread. It had the perfect amount of cheese and garlic. The bread wasn't hard either just right. For my main entree I went with chicken broccoli fettuccine Alfredo. I definitely made the right choice. It tasted delicious! To top it off they include dessert with dinner. :),"[101, 23330, 1142, 1282, 1108, 1177, 1186, 14820, 119, 146, 2802, 1103, 12647, 26883, 6198, 25150, 24861, 9553, 8162, 119, 1135, 1125, 1103, 3264, 2971, 1104, 9553, 1105, 24861, 119, 1109, 8162, 1445, 112, 189, 1662, 1719, 1198, 1268, 119, 1370, 1139, 1514, 4035, 14750, 146, 1355, 1114, 9323, 9304, 26108, 2646, 175, 5912, 15072, 1673, 21269, 119, 146, 5397, 1189, 1103, 1268, 3026, 119, 1135, 12876, 13108, 106, 1706, 1499, 1122, 1228, 1152, 1511, 20392, 1114, 4014, 119, 131, 114, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]"


In [16]:
### 数据抽样
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(50000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(50000))

In [17]:
##微调训练配置
### 加载BERT模型
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased",num_labels=5)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
### 训练超参数（TrainingArguments）
from transformers import TrainingArguments

model_dir = "models/bert-base-cased-finetune-yelp"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=model_dir,
                                  per_device_train_batch_size=16,
                                  num_train_epochs=5,
                                  logging_steps=100)

In [19]:
## 打印完整的超参数
print(training_args)

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fs

In [20]:
## 训练过程中的指标评估（Evaluate)
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [21]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [22]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=30)

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [24]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.846300,0.851991,0.631080
2,0.647700,0.856956,0.646060
3,0.397300,0.998499,0.647660


TrainOutput(global_step=9375, training_loss=0.6950175591023763, metrics={'train_runtime': 4702.9888, 'train_samples_per_second': 31.895, 'train_steps_per_second': 1.993, 'total_flos': 3.94677213696e+16, 'train_loss': 0.6950175591023763, 'epoch': 3.0})

In [30]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(50000))

In [31]:
trainer.evaluate(small_test_dataset)

{'eval_loss': 0.9984992146492004,
 'eval_accuracy': 0.64766,
 'eval_runtime': 437.6935,
 'eval_samples_per_second': 114.235,
 'eval_steps_per_second': 14.279,
 'epoch': 3.0}

In [27]:
trainer.save_model(model_dir)

In [28]:
trainer.save_state()

In [29]:
trainer.model.save_pretrained("./")